# Draft: Aligning and Resampling Spectra

***

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data

In [2]:
%matplotlib inline
import numpy as np
from specutils import Spectrum1d
from specutils import manipulation
from astropy.wcs.utils import pixel_to_pixel
from astropy.modeling import models, fitting
from astropy.convolution import convolve_fft
import astropy.units as u
from scipy.interpolate import interp1d
from jdaviz import SpecViz
from webbpsf import NIRSpec, display_psf

Sections:
 - Use specviz to identify good regions for fitting
 - Same instrument mode (dithers, etc.)
 - Same instrument, different modes (filters/grisms)
 - Different instruments (maybe?)

## Introduction
Write a short introduction explaining the purpose of the notebook. Link to any background materials/resources that may be useful to the reader to provide additional context.

### Defining terms
Be sure to define any terms/common acronyms at the end of your introduction that your audience may not know. If you're using some kind of domain-specific astronomical symbol or unusual mathematical concept, make sure you define it (e.g. in its mathematical form) and link to any definitions (literature/Wikipedia etc.).

***

## Loading data
Need example data for this, with some variety for the different tools:
1. Reference spectrum
2. Same configuration as spectrum 1 (different dither?)
3. Same instrument, different configuration as spectrum 1.
4. Different instrument as spectrum 1.

Each spectra then stored as `specutils.Spectrum1D`

Where possible (if the code supports it): use code examples that use Jupyter to show what's in the data. For example, if you are showing an object such as a Table, display a preview of the table:

In [ ]:
yourProd[0:5]

In [ ]:
# Download the products
Observations.download_products(yourProd, mrp_only=False, cache=False)

## File information

Explain pertinent details about the file you've just downloaded. 

# Resampling & Aligning Toolbox

## Use specviz to identify good regions for fitting

In [ ]:
specviz = SpecViz()
specviz.app()

In [ ]:
specviz.load_spectrum(spec1)
specviz.load_spectrum(spec2)
specviz.load_spectrum(spec3)

## Combining spectra from the same instrument configuration

I.e., different dithers. Don't need to convolve because the PSF is the same. 

We're going to skip aligning the spectra via individual lines because we may not have enough signal before combining to be able to identify them.

1. Do everything with specutils (via WCS)
2. Do it by hand
    1. align spectra via fitting
    2. Resample & combine

In [ ]:
# Spectral registration, resampling, & combination via specutils
# --------------------------------------------------------------
# This is very straightforward, and is the recommended method in general.
# Since we have WCS data for both, we can use a resampler from
# specutils.manipulation to do the heavy lifting.

fluxcon = manipulation.FluxConservingResampler()
ref_dispersion_grid = spec1.spectral_axis
spec2_reg_su = fluxcon(spec2, ref_dispersion_grid)

#plot spec1, spec2 before, spec2 after

In [ ]:
# Spectral registration via fitting
# ---------------------------------
# What we want is a composite model which transforms the original pixel coordinates
# to registered coordinates (as pixel_to_pixel above). Since we're not identifying
# individual spectral features, we have to use astropy.modeling in an unusual way.

# We'll assume that the transformation is a 3rd-degree polynomial for this example,
# but different datasets may require different functions. We define a custom model
# that takes the flux from spectra 2 and resamples it onto the grid of spectra 1,
# using our polynomial coefficients for the pixel transformation. (This is a preview
# of the next subsection, but here we're not worrying about flux conservation.)

# We want to fit to a (median-)normalized spectrum, since we're just looking to fit
# to general shape and features. For a better fit, use specutils to identify good
# regions for fitting in the previous section.

norm1 = spec1.flux / np.abs(np.median(spec1.flux))
norm2 = spec2.flux / np.abs(np.median(spec2.flux))

@models.custom_model
def pixel_transform(px, c0=0., c1=1., c2=0., c3=0.):
    new_px = np.zeros_like(px) + c0.
    new_px += c1 * px
    new_px += c2 * px * px
    new_px += c3 * px * px * px
    resampled = interp1d(new_px, norm2)(pix1)
    return resampled

fitter = fitting.LevMarLSQFitter() #nonlinear
pix2_reg_fit = fitter(pixel_transform, pix2_original, norm1)

#plot spec1, spec2 before, spec2 after

## Resample & Combine Spectra

In [ ]:
# Resample & Combine
# ------------------


# options - use fluxcon (in which case why aren't we just
# using specutils for everything? maybe we don't have WCS?)
# or roll our own interpolator (in which case we're reinventing
# the wheel...)

## Matching PSFs

Used for combining spectra from the same instrument, different modes. Once we have matching PSFs, then we can use the previous section.

In [ ]:
# Make sure WebbPSF works
try:
    instrument = NIRSpec()
except OSError:
    # assume that WebbPSF data files have not been downloaded
    import tarfile, sys
    print("Downloading WebbPSF data files.")
    webb_url = "https://stsci.box.com/shared/static/qcptcokkbx7fgi3c00w2732yezkxzb99.gz"
    webb_file = os.path.join('.', "webbpsf-data-0.9.0.tar.gz")
    urllib.request.urlretrieve(webb_url, webb_file)
    print("Extracting into ./webbpsf-data ...")
    tar = tarfile.open(webb_file)
    tar.extractall()
    tar.close()
    os.environ["WEBBPSF_PATH"] = os.path.join(".","webbpsf-data")

**Developer note:** this section is only really useful if NIRSpec spectral science filters are implemented in WebbPSF... and we really need to use WebbPSF since the NIRSpec PSF is complicated.

In [ ]:
# Create a convolution kernel
# ---------------------------
# We want to smooth the higher-resolution spectrum to match the lower resolution.
# We can make use of Fourier space for this. We'll assume that the reference spectrum
# (spec1) has the lower resolution.

# Luckily, making just two monochromatic kernels with WebbPSF is relatively fast.

instrument1, instrument3 = NIRSpec(), NIRSpec()
# Note that currently only TA filters are supported.
instrument1.filter = model1.meta.filter
instrument3.filter = model3.meta.filter
mono_wave1 = np.median(spec1.spectral_axis).to(u.m) #WebbPSF requires wavelength in meters
mono_wave3 = np.median(spec3.spectral_axis).to(u.m)

# Since we don't know the original 2D FOV, we'll just use the default and coadd.
mono_psf1 = instrument.calc_psf(monochromatic=mono_wave1).sum(axis=0)
mono_psf3 = instrument.calc_psf(monochromatic=mono_wave3).sum(axis=0)

# Now we combine them in Fourier space...
fpsf1 = np.fft.fft(mono_psf1)
fpsf3 = np.fft.fft(mono_psf3)
kernel = np.fft.ifft(fpsf1 / fpsf3)

# ...and convolve with spectrum 3
# Note that there are many options for convolve_fft, particularly in how
# it deals with nan values. Check the documentation for the best configuration
# for a particular dataset.
smoothed_spec3 = convolve_fft(spec3.flux, kernel)

# plot initial specs, smoothed spec for comparison

In [ ]:
# Account for wavelength-varying PSF. 
# -----------------------------------
# We can generate psfs with WebbPSF as per the optimal extraction
# notebook (and PSF matching from the previous step).

# adapted from opt extract notebook
wave1 = spec1.spectral_axis.to(u.m)
wave3 = spec3.spectral_axis.to(u.m)
nwave1, nwave3 = wave1.size, wave3.size
psf_wave1 = np.linspace(wave1[0], wave1[-1], num=10)
psf_wave3 = np.linspace(wave3[0], wave3[-1], num=10)
cube1_hdul = instrument.calc_datacube(psf_wave1) #the output is a HDUList
cube3_hdul = instrument.calc_datacube(psf_wave3)
psf_cube1, psf_cube3 = cube1_hdul[1].data, cube3_hdul[1].data
psf_x = psf_y = np.arange(48)
out_x, out_y = np.meshgrid(psf_x, psf_y, indexing='ij')
interpolator1 = RegularGridInterpolator((psf_wave1, psf_x, psf_y), psf_cube1, method='linear')
interpolator3 = RegularGridInterpolator((psf_wave3, psf_x, psf_y), psf_cube3, method='linear')
interp_psf1 = interpolator1((wave1, out_x, out_y)) # grab both PSFs at the spec3 wavelength locations
interp_psf3 = interpolator3((wave1, out_x, out_y)) # so we can combine them as per the previous step

var_kernel = np.zeros_like(interp_psf3)
for i, (w1, w3) in enumerate(zip(interp_psf1, interp_psf3)):
    fpsf1 = np.fft.fft2(w1)
    fpsf3 = np.fft.fft2(w3)
    k = np.fft.ifft2(fpsf1 / fpsf3)
    var_kernel[i] = k

var_kernel = var_kernel.sum(axis=1) 

In [ ]:
# Now we convolve with the spatially-varying kernel by
# iterating over each spectral flux element, multiplying
# by the kernel at that wavelength, and adding it to the
# appropriate bins in the smoothed spectrum array.

dy = psf_y - psf_y[psf_y.size//2] #offsets along the dispersion axis

var_smoothed_spec3 = np.zeros_like(spec3.flux)
for i, spel in enumerate(spec3.flux):
    window = i + dy
    in_range = (window >= 0) & (window < nwave3)
    vkern = var_kernel[i, psf_y[in_range]]
    var_smoothed_spec3[window[in_range]] += spel * vkern
    
# plot initial specs, smoothed spec, non-varying smoothed spec for comparison

# Full example

*Actually writing this will require some example data... but it'll be fast because we're basically just copy/pasting from above.*

1. Identify regions
2. Spectral registration
3. Match PSFs
4. Resample & combine

## Exercises
Most notebooks are trying to convey _some_ information to their reader. It's often a good idea, if possible, to include in the notebook something where there is not an included answer so that the reader can cement in their mind how whatever it is they were supposed to learn from that notebook. If you do have exercise(s), be sure to leave a blank code cell underneath to show the user that it's meant for them to try it out right there. You may also want to include a "solutions" notebook next to your main notebook for the user to look at after they have finished their attempt.

## Aditional Resources
While this isn't always necessary, sometimes you want to provide some more resources for the reader who wants to learn something beyond what's in the notebook. Sometimes these don't exist, but if they do, it's good to put them at the end to give the reader somewhere else to go. Usually a list of links using markdown bullet-plus-link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/exo/ExoMast/html/exomast.html)

## About this notebook
Let the world know who the author of this great notebook is! If possible/appropriate, include a contact email address for users who might need support (e.g. archive@stsci.edu)

**Author:** Jessie Blogs, Archive Scientist.  
**Updated On:** YYYY-MM-DD

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 